In [ ]:
import holoviews as hv
import numpy as np
hv.extension('bokeh','matplotlib')
class dots:
    def __init__(self,vec,**kw):
        """
        vec - Vector (sepVector or gieeVector)
        
        Optional:
        
          dots - Whether or not to draw dots [True]
          
          dotSize[9] Size of dots
          dotColor[black] Color of dot
          spikeWidth[1]  Width of line
          spikeColor[black] Spiek color
        
        
        """
        remove=["dotSize","dotColor","spikeWidth","spikeColor"]
        
        self._spikeOpts=kw
        self._dotOpts={}
        
        if "dotSize" in kw:
            self._dotOpts["size"]=kw["dotSize"]
        else:
            self._dotOpts["size"]=8
            
        if "dotColor" in kw:
            self._dotOpts["color"]=kw["dotColor"]
        else:
            self._dotOpts["color"]="black"
        
        if "spikeWidth" in kw:
            self._spikeOpts["line_width"]=kw["spikeWidth"]
        else:
            self._spikeOpts["line_width"]=3
        
        if "spikeColor" in kw:
            self._spikeOpts["color"]=kw["spikeColor"]
        else:
            self._spikeOpts["color"]="black"
        
        if len(vec.getHyper().axes)!=1:
            raise Exception("Can only handle 1-D functions")
            
            
        for a in remove:
            if a in self._spikeOpts:
                del self._spikeOpts[a]
                
        self._ax=vec.getHyper().axes[0]
        
        self._spk=np.ndarray(shape=(self._ax.n,2))
        v=vec.getNdArray()
        mn=v[0]
        mx=v[0]
        
        for i in range(self._ax.n):
            self._spk[i,0]=self._ax.o+i*self._ax.d
            self._spk[i,1]=v[i]
            if v[i] < mn:
                mn=v[i]
            if v[i] >mx:
                mx=v[i]
            
        
        if "xlim" in kw:
            self._spikeOpts["xlim"]=kw["xlim"]
        else:
            self._spikeOpts["xlim"]=(self._ax.o-self._ax.d/2.,self._ax.o+self._ax.d*(self._ax.n-.5))
        
        if "ylim" in kw:
            self._spikeOpts["ylim"]=kw["ylim"]
        else:
            delta=mx-mn
            self._spikeOpts["ylim"]=(mn-delta/10.,mx+delta/10.)

        self._dotOpts["ylim"]=self._spikeOpts["ylim"]
        self._dotOpts["xlim"]=self._spikeOpts["xlim"]



        self.scat=hv.Scatter(self._spk).opts(**self._dotOpts)
        self.spik=hv.Spikes(self.scat).opts(**self._spikeOpts)
        if dots:
            self._img=self.scat*self.spik
        else:
            self._img=self.spik
    def image(self):
        """Return image"""
        return self._img
    


In [ ]:
import giee
import BoxFilter
import Hypercube
import Tconv 

vec=np.ndarray(16)
vec.fill(1.)
mod=giee.vector(Hypercube.hypercube(ns=[16]),arr=vec)
vec.fill(0.)
filtA=giee.vector(Hypercube.hypercube(ns=[16]),arr=vec)
dat=mod.clone()
flt=BoxFilter.filter([2],[0],False)
ff=flt.getNdArray()
ff[0]=1
ff[1]=-.5
ff=filtA.getNdArray()
ff[0]=1
ff[1]=-.5

conv=Tconv.tconvM(flt,mod,dat)
conv.forward(False,mod,dat)

a=dots(mod,xaxis=None,yaxis=None,width=600,height=100,title="Input",ylim=(-1,2))
b=dots(filtA,xaxis=None,yaxis=None,width=600,height=100,title="Filter",ylim=(-1,2))
c=dots(dat,xaxis=None,yaxis=None, width=600,height=100,title="Output",ylim=(-1,2))

hv.Layout(a.image()+b.image()+c.image()).cols(1)


In [ ]:
hv.help(hv.Spikes)

In [ ]:
!ls /opt/SEP/lib/python